# Population density and COVID-19 mortality
Carl Higgs, Deepti Adlakha, Jim Sallis; 9 August 2022


In [1]:
import pandas as pd

# Preliminary sampling for tertiles of population density


1. Identify the 100 largest urban centres in each region
2. Create tertiles of population density in each region
3. Randomly sample 5 cities  (or as per original suggestion, use the top 5-most dense)
4. Locate COVID-19 mortality data for these cities, where available; else, re-draw a new sample from the relevant continent-specific density tertile
5. Create a scatterplot of results

In the below the following data from the Global Human Settlements Urban Centres Database are retained:

Variable  | Description
----------|------------
GRGN_L1   | Major Geographical Region (UNDESA, 2018b), according to the classification of the main country
CTR_MN_ISO| The ISO 3 code of the main country, i.e., the country within which borders the majority of the area of the Urban Centre is located
CTR_MN_NM | The name of the main country, i.e., the country within which borders the majority of the area of the Urban Centre is located
UC_NM_MN  | The main name of the Urban Centre (the country ISO 3 is declared within ‘[]’, to support the cross border entities);
AREA      | Area in square kilometres
B15       | Total built-up area in 2015 calculated within the spatial domain of the Urban Centre of 2015, expressed in square kilometres
P15       | Total resident population in 2015

In [2]:
# Get the GHS UCDB data
# zip file including multiple formats from
# https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/GHS_STAT_UCDB2015MT_GLOBE_R2019A/V1-2/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.zip
df = pd.read_csv('./GHS/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.csv')
covariates = ['GRGN_L1','CTR_MN_NM','UC_NM_MN','AREA','B15','P15']
df = df[covariates]

# only analyse named urban centres (because some small ones in Oceania in particular are not named)
# and set a minimum area of at least 10 square kilometres to mitigate extreme densities 
# for smaller, low population settlements
df = df.loc[(df['UC_NM_MN'].isna()==False) & (df['AREA']>20)]

# calculate population per sqkm in 2015
# using B15, built up area, as more closely matches values eg for Dhaka in Adlakha et al 2020 paper
# however, it is still lower (eg Dhaka approximately 28k instead of 45k)
# difference no doubt largely because targetting a time point 5 years earlier
df['population_density_2015'] = df['P15']/df['B15']
# for display purposes, make population estimate an integer
df['P15'] = df['P15'].astype('int')

df

,GRGN_L1,CTR_MN_NM,UC_NM_MN,AREA,B15,P15,population_density_2015
0,Northern America,United States,Honolulu,185,80.647377,512853,6359.210748
1,Oceania,French Polynesia,Papeete,42,14.493433,91521,6314.661586
2,Northern America,United States,Santa Maria,55,42.000805,123181,2932.831529
3,Northern America,United States,Monterey,48,27.759470,67772,2441.411486
4,Northern America,United States,Santa Barbara,60,38.101749,114753,3011.755416
...,...,...,...,...,...,...,...
13129,Oceania,New Zealand,Hillcrest,115,56.726643,219043,3861.378420
13130,Oceania,New Zealand,Tauranga,70,31.697439,84583,2668.471856
13132,Oceania,Solomon Islands,Honiara,23,4.069641,73669,18102.255748
13133,Oceania,New Caledonia,Nouméa,27,12.111768,70631,5831.666991


In [3]:
# Identify the 100 largest urban centres in each region
selected = df.groupby('GRGN_L1')\
  .apply(lambda x : x.sort_values(by = 'AREA', ascending = False)\
                    .head(100)).reset_index(drop = True)

In [4]:
# all regions have 100 records, except for Oceania, which has 6
selected.groupby('GRGN_L1')['UC_NM_MN'].count()

GRGN_L1
Africa                             100
Asia                               100
Europe                             100
Latin America and the Caribbean    100
Northern America                   100
Oceania                             40
Name: UC_NM_MN, dtype: int64

In [5]:
# Create region specific tertiles of population density for the 100 largest cities in each region which were identified
selected['region_density_tertile'] = selected.groupby(['GRGN_L1'])['population_density_2015'].transform(
                     lambda x: pd.qcut(x, 3, labels=['Q1','Q2','Q3'])
)
selected

,GRGN_L1,CTR_MN_NM,UC_NM_MN,AREA,B15,P15,population_density_2015,region_density_tertile
0,Africa,South Africa,Johannesburg,1638,900.362427,6516134,7237.234741,Q1
1,Africa,Egypt,Cairo,1585,637.769348,19734085,30942.355237,Q3
2,Africa,Nigeria,Lagos,1196,813.717163,11575042,14224.896629,Q2
3,Africa,Ghana,Accra,846,494.896210,4412617,8916.248266,Q1
4,Africa,Nigeria,Onitsha,781,261.669800,3150801,12041.136163,Q1
...,...,...,...,...,...,...,...,...
535,Oceania,Australia,River Glen Village,32,16.558352,56992,3441.911782,Q2
536,Oceania,Australia,Lindum,28,16.999882,53260,3132.995184,Q2
537,Oceania,New Caledonia,Nouméa,27,12.111768,70631,5831.666991,Q3
538,Oceania,New Zealand,Palmerston North,25,14.775366,51492,3485.018617,Q3


In [6]:
# Sample 5 cities from each region specific tertile
#sdf = df.groupby(['GRGN_L1','region_density_tertile']).sample(3,random_state=20220809)

# Top 3
# NOTE: this approach excludes cities on islands in Oceania other than Australia and NZ
# the benefit of this is some smaller cities like Madang and Kokopo in PNG have ridiculously high density estimates
# approaching 60,000 persons per sqkm.
sample = selected.groupby(['GRGN_L1','region_density_tertile']).head(3).sort_values(['GRGN_L1','region_density_tertile'])
sample

,GRGN_L1,CTR_MN_NM,UC_NM_MN,AREA,B15,P15,population_density_2015,region_density_tertile
0,Africa,South Africa,Johannesburg,1638,900.362427,6516134,7237.234741,Q1
3,Africa,Ghana,Accra,846,494.896210,4412617,8916.248266,Q1
4,Africa,Nigeria,Onitsha,781,261.669800,3150801,12041.136163,Q1
2,Africa,Nigeria,Lagos,1196,813.717163,11575042,14224.896629,Q2
5,Africa,Angola,Luanda,771,463.673126,6786991,14637.447828,Q2
6,Africa,Sudan,Khartoum,750,264.521667,5824720,22019.825176,Q2
1,Africa,Egypt,Cairo,1585,637.769348,19734085,30942.355237,Q3
13,Africa,Nigeria,Amaigbo,510,61.440853,1713470,27888.133139,Q3
23,Africa,Morocco,Casablanca,393,157.085449,3985659,25372.557616,Q3
101,Asia,Japan,Tokyo,5318,3664.912354,33028731,9012.147641,Q1


The above is a test run of the discussed sampling method.  There are some awkward aspects, in particular setting a minimum area.  The selection of top 3 vs random 3 cities within regional density tertiles is important --- top3 more or less excludes cities on small islands of Oceania (although linkage with Covid data for these could be problematic, so could be useful). 

Notwithstanding the above -- it is a selection of cities which may now be linked with Covid mortality data, and potentially custom updating of the density statistics.

Once we have Covid mortality data linked, the below are example sketches of how the scatterplots could be produced (noting the current Y axis of 'AREA' makes no sense -- just a placeholder).

In [7]:
# see the assumptions above with GHS data export, this may not be what you're after either
sample.to_csv('GHS - sampled_city_population_density_tertiles_by_region.csv', encoding='utf-8')

# Covid data linkage

In [18]:
# Read in JHU data from
# https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/08-13-2022.csv
# and print summary of cities by country
df = pd.read_csv('./covid-data/JHU - CSSEGISandData- COVID19 - 08-13-2022.csv')
df = df.loc[~(df['Province_State'].isna())]
df.groupby(['Country_Region'])['Province_State'].count()


Country_Region
Australia            8
Belgium             12
Brazil              27
Canada              16
Chile               17
China               34
Colombia            34
Denmark              2
France              11
Germany             17
India               37
Italy               21
Japan               49
Malaysia            17
Mexico              33
Netherlands         17
New Zealand          1
Pakistan             7
Peru                26
Russia              83
Spain               20
Sweden              21
US                3279
Ukraine             28
United Kingdom      18
Name: Province_State, dtype: int64

In [23]:
# test to check for direct match
# this is naive -- there may be a match which doesn't work due to spelling
sample[sample['UC_NM_MN'].isin(df['Province_State'])]

,GRGN_L1,CTR_MN_NM,UC_NM_MN,AREA,B15,P15,population_density_2015,region_density_tertile
101,Asia,Japan,Tokyo,5318,3664.912354,33028731,9012.147641,Q1
103,Asia,China,Shanghai,3318,2106.193359,24472119,11619.122714,Q1
200,Europe,Russia,Moscow,1882,1218.433960,14077364,11553.653975,Q3
303,Latin America and the Caribbean,Brazil,Rio de Janeiro,1367,795.496094,9798104,12316.974122,Q2
304,Latin America and the Caribbean,Peru,Lima,876,390.686066,9265580,23716.178571,Q3
401,Northern America,United States,New York,5384,3678.075928,15950674,4336.689781,Q3


In [25]:
# the following cities are in country's with cities present in the JHU data, so at least some could be expected to match
# and may require a similarity match
sample[sample['CTR_MN_NM'].isin(df['Country_Region'])]

,GRGN_L1,CTR_MN_NM,UC_NM_MN,AREA,B15,P15,population_density_2015,region_density_tertile
101,Asia,Japan,Tokyo,5318,3664.912354,33028731,9012.147641,Q1
103,Asia,China,Shanghai,3318,2106.193359,24472119,11619.122714,Q1
105,Asia,Japan,Osaka [Kyoto],3158,2113.337891,15692797,7425.597894,Q1
100,Asia,China,Guangzhou,6622,2771.371826,40589878,14646.132186,Q2
106,Asia,India,Kolkata,2817,1218.220703,21620289,17747.432156,Q2
110,Asia,India,Delhi [New Delhi],2474,1212.729614,26658713,21982.405051,Q3
203,Europe,Germany,Dortmund,1315,802.928833,3443652,4288.864397,Q1
209,Europe,United Kingdom,Birmingham,668,457.201355,2426862,5308.082567,Q1
210,Europe,Netherlands,Rotterdam [The Hague],658,419.476685,1913888,4562.561563,Q1
204,Europe,Italy,Naples,899,574.565613,3167668,5513.153583,Q2


In [28]:
# as per the datasets columnb names -- these are province/state statistics for other countries 
# --- so not necessarily for cities, other than US
df[df['Country_Region'].isin(sample['CTR_MN_NM'])]

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
9,NaN,NaN,Australian Capital Territory,Australia,2022-08-14 04:20:55,-35.4735,149.0124,198193,110,NaN,NaN,"Australian Capital Territory, Australia",46295.958888,0.055501
10,NaN,NaN,New South Wales,Australia,2022-08-14 04:20:55,-33.8688,151.2093,3336902,4525,NaN,NaN,"New South Wales, Australia",41104.976595,0.135605
11,NaN,NaN,Northern Territory,Australia,2022-08-14 04:20:55,-12.4634,130.8456,93934,59,NaN,NaN,"Northern Territory, Australia",38246.742671,0.062810
12,NaN,NaN,Queensland,Australia,2022-08-14 04:20:55,-27.4698,153.0251,1574198,1757,NaN,NaN,"Queensland, Australia",30773.101359,0.111612
13,NaN,NaN,South Australia,Australia,2022-08-14 04:20:55,-34.9285,138.6007,736959,753,NaN,NaN,"South Australia, Australia",41956.105892,0.102177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3997,NaN,NaN,"Saint Helena, Ascension and Tristan da Cunha",United Kingdom,2022-08-14 04:20:55,-7.9467,-14.3559,4,0,NaN,NaN,"Saint Helena, Ascension and Tristan da Cunha, ...",70.658894,0.000000
3998,NaN,NaN,Scotland,United Kingdom,2022-08-14 04:20:55,56.4907,-4.2026,2079366,12389,NaN,NaN,"Scotland, United Kingdom",38060.622701,0.595807
3999,NaN,NaN,Turks and Caicos Islands,United Kingdom,2022-08-14 04:20:55,21.6940,-71.7979,6338,36,NaN,NaN,"Turks and Caicos Islands, United Kingdom",16369.647193,0.568003
4000,NaN,NaN,Unknown,United Kingdom,2022-08-14 04:20:55,NaN,NaN,0,0,NaN,NaN,"Unknown, United Kingdom",NaN,NaN


In [31]:
# Even the US data is problematic, unless grouped by province....

sample.loc[(sample['CTR_MN_NM']=='US') &
           (sample['UC_NM_MN'].isin(df['Admin2']))]

,GRGN_L1,CTR_MN_NM,UC_NM_MN,AREA,B15,P15,population_density_2015,region_density_tertile


In [39]:
# eg as per New York, its disaggregated into smaller Admin2 levels
df.loc[df['Province_State']=='New York']

# perhaps the assumption we can do is 'Province_State' is analogous to a city?
for country in sample[sample['CTR_MN_NM'].isin(df['Country_Region'])]['CTR_MN_NM'].unique():
    print(f'\n{country}')
    print(df.loc[df['Country_Region']==country]['Province_State'].to_list())


Japan
['Aichi', 'Akita', 'Aomori', 'Chiba', 'Ehime', 'Fukui', 'Fukuoka', 'Fukushima', 'Gifu', 'Gunma', 'Hiroshima', 'Hokkaido', 'Hyogo', 'Ibaraki', 'Ishikawa', 'Iwate', 'Kagawa', 'Kagoshima', 'Kanagawa', 'Kochi', 'Kumamoto', 'Kyoto', 'Mie', 'Miyagi', 'Miyazaki', 'Nagano', 'Nagasaki', 'Nara', 'Niigata', 'Oita', 'Okayama', 'Okinawa', 'Osaka', 'Port Quarantine', 'Saga', 'Saitama', 'Shiga', 'Shimane', 'Shizuoka', 'Tochigi', 'Tokushima', 'Tokyo', 'Tottori', 'Toyama', 'Unknown', 'Wakayama', 'Yamagata', 'Yamaguchi', 'Yamanashi']

China
['Anhui', 'Beijing', 'Chongqing', 'Fujian', 'Gansu', 'Guangdong', 'Guangxi', 'Guizhou', 'Hainan', 'Hebei', 'Heilongjiang', 'Henan', 'Hong Kong', 'Hubei', 'Hunan', 'Inner Mongolia', 'Jiangsu', 'Jiangxi', 'Jilin', 'Liaoning', 'Macau', 'Ningxia', 'Qinghai', 'Shaanxi', 'Shandong', 'Shanghai', 'Shanxi', 'Sichuan', 'Tianjin', 'Tibet', 'Unknown', 'Xinjiang', 'Yunnan', 'Zhejiang']

India
['Andaman and Nicobar Islands', 'Andhra Pradesh', 'Arunachal Pradesh', 'Assam', '

The more I look at the JHU data, the less I think it will be appropriate for this purpose of representing cities.  For some cities, the state aggregate may be approximately correct.  For example, Goiânia is  capital of Goias state in Brazil, which is effectively the same expanse as the city), but for most this would be doubtful --- and even for the US, the Admin2 data is much smaller -- e.g. for New York State, the burrough of Queens in New York City, but also Seneca County which is nowhere near, and is closer to Toronto, Canada; representing metropolitan areas by grouping on province/state name where these happen to match is unlikely to be valid. 

# Example plots

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# set up for plotting
plt.rcParams.update({'figure.figsize':(10,8), 'figure.dpi':100})
def annotate_points(x,y,t,offset=100,df=None, **kwargs):
    ax = plt.gca()
    if df is None:
        data = kwargs.pop('data')
    else:
        data=df.copy()
    for i,row in data.iterrows():
        ax.annotate(row[t], xy=(row[x],row[y]+offset))

markers=['x','o','^']
        
# scatter plot of y with regard to x by col with levels of hue
g = sns.FacetGrid(sample, col='GRGN_L1',col_wrap=1,aspect=4,
                  #hue='region_density_tertile', 
                  sharex=False,
                 sharey=False)
g.map(sns.scatterplot,'population_density_2015','AREA')
#g = sns.lmplot(x="population_density_2015", 
#               y="AREA", 
#               #hue="region_density_tertile", 
#               col="GRGN_L1",
#               data=sdf, 
#               height=3, 
#               aspect=2,
#               col_wrap=1, 
#               sharex=False,
#               sharey=False,
#               legend=True, 
#               #markers=markers,
#               fit_reg=False)

g.map_dataframe(annotate_points,'population_density_2015','AREA','UC_NM_MN')
g.set_titles('{col_name}',fontdict= { 'fontsize': 24, 'fontweight':'bold', 'horizontalalignment':'left','x':0})
g.fig.subplots_adjust(top=0.9) # adjust the Figure in rp
g.fig.suptitle('Example scatterplot of Y by population density\n(noting mortality data not yet linked and densities not updated)')
g.set_axis_labels("Population density (km²)", "Placeholder for mortality")


In [ ]:
markers=['^']*6
g = sns.lmplot(x="population_density_2015", 
               y="AREA", 
               hue="GRGN_L1",
               data=sample, 
               aspect=2,
               legend=True, 
               fit_reg=False,
               logx=True,
               ci=95,
              truncate=True,
              markers=markers)
sns.regplot(x="population_density_2015", 
            y="AREA", 
            data=sample, 
            scatter=False, 
            ax=g.axes[0, 0])
g.map_dataframe(annotate_points,'population_density_2015','AREA','UC_NM_MN',df=sample.copy())
g.fig.subplots_adjust(top=0.9) # adjust the Figure in rp
g.fig.suptitle('Example scatterplot of Y by population density\n(noting mortality data not yet linked and densities not updated)')
g.set_axis_labels("Population density (km²)", "Placeholder for mortality")
g.legend.set_title('Region',prop={'weight':'bold'})
sns.move_legend(g, "upper right")

## not implemented attempt at changing x axis scale
#ticks = [1000,2000,4000,8000,16000]
#g.ax.set_xticks(ticks)
#plt.xscale('log')
#g.ax.set_xticklabels(ticks)
